In [ ]:
%pip install -e ..
%restart_python

In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent / 'src'))

In [ ]:
import argparse
import sys
from loguru import logger
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

from honeywell.serving.model_serving import ModelServing

# ------------------------------------------------------------------------------
# Databricks Context
# ------------------------------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:

# # ---------------------------------------------------------
# # Args
# # ---------------------------------------------------------
# parser = argparse.ArgumentParser(description="Deploy or update Databricks model serving endpoint")

# parser.add_argument(
#     "--model_name",
#     type=str,
#     default="honeywell_mlops_dev.honeywell.ev_battery_charging_model_basic",
#     # required=True,
#     help="Fully-qualified UC model name (e.g. catalog.schema.model_name)",
# )

# parser.add_argument(
#     "--env",
#     type=str,
#     default="dev",
#     # required=True,
#     choices=["dev", "staging", "prod"],
#     help="Deployment environment",
# )

# parser.add_argument(
#     "--upstream_task_key",
#     # type=str,
#     default="check_model_version",
#     help="Databricks Jobs taskKey that produced model_version",
# )
# parser.add_argument(
#     "--model_version",
#     default="4",
#     help="Model version to deploy (manual fallback)",
# )

# parser.add_argument(
#     "--wait_for_ready",
#     # default=True,
#     action="store_true",
#     help="Wait until serving endpoint becomes READY",
# )

# parser.add_argument(
#     "--timeout_sec",
#     type=int,
#     default=900,
#     help="Timeout (seconds) to wait for endpoint readiness",
# )

# # args = parser.parse_args()
# args, unknown = parser.parse_known_args()
# if unknown:
#     logger.info("Ignoring unknown args: %s", unknown)


In [ ]:
# ---------------------------------------------------------
# Widgets (Notebook Arguments)
# ---------------------------------------------------------
# Define the widgets (slots for your parameters)
dbutils.widgets.text("model_name", "honeywell_mlops_dev.honeywell.ev_battery_charging_model_basic")
dbutils.widgets.dropdown("env", "dev", ["dev", "staging", "prod"])
dbutils.widgets.text("model_version", "4")
dbutils.widgets.text("timeout_sec", "900")
dbutils.widgets.dropdown("wait_for_ready", "false", ["true", "false"])

# Retrieve the values into variables
# Note: Widgets always return strings, so we cast numbers and booleans manually
arg_model_name = dbutils.widgets.get("model_name")
arg_env = dbutils.widgets.get("env")
arg_model_version = dbutils.widgets.get("model_version")
arg_timeout_sec = int(dbutils.widgets.get("timeout_sec"))
arg_wait_for_ready = dbutils.widgets.get("wait_for_ready").lower() == "true"

logger.info(f"Deploying {arg_model_name} version {arg_model_version} to {arg_env}")


In [ ]:

# ---------------------------------------------------------
# Spark + DBUtils
# ---------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:

# ---------------------------------------------------------
# Load model_version from upstream task
# ---------------------------------------------------------
logger.info("Fetching model version from upstream task (if any)...")

# model_version = dbutils.jobs.taskValues.get(
#     taskKey=args.upstream_task_key,
#     key="model_version",
#     debugValue=None,   # must be provided outside job context
# )

# if model_version:
#     logger.info("✅ Model version from upstream task: {}", model_version)
# else:
#     logger.info("ℹ️ No upstream task value found. Falling back to manual arg.")
# model_version = args.model_version

model_version = arg_model_version

if not model_version:
    logger.error("❌ No model_version available from upstream task or args.")
    sys.exit(1)

logger.info("✅ Final model version to deploy: {}", model_version)

In [ ]:

# ---------------------------------------------------------
# Clean serving endpoint name (Databricks-safe)
# ---------------------------------------------------------
def make_endpoint_name(model_name: str, env: str) -> str:
    """
    Generate a Databricks-compliant serving endpoint name.
    """
    clean = model_name.lower()
    clean = clean.replace(".", "-").replace("_", "-")

    endpoint = f"{clean}-serving-{env}"

    # Databricks max length = 63 chars
    if len(endpoint) > 63:
        logger.warning(
            "Endpoint name too long ({} chars). Truncating to 63.",
            len(endpoint),
        )
        endpoint = endpoint[:63]

    return endpoint


# endpoint_name = make_endpoint_name(args.model_name, args.env)
endpoint_name = make_endpoint_name(arg_model_name, arg_env)
logger.info("Using endpoint name: {}", endpoint_name)

In [ ]:

# ---------------------------------------------------------
# Guardrails for production safety
# ---------------------------------------------------------
# if args.env == "dev" and not model_version:
if arg_env == "dev" or arg_env == "acc" and not model_version:
    logger.error("❌ Refusing to deploy to DEV or ACC without a model_version.")
    sys.exit(1)

# logger.info("Environment: {}", args.env)
# logger.info("Model name: {}", args.model_name)
logger.info("Environment: {}", arg_env)
logger.info("Model name: {}", arg_model_name)

In [ ]:

# ---------------------------------------------------------
# Initialize Serving Manager
# ---------------------------------------------------------
model_serving = ModelServing(
    # model_name=args.model_name,     # UC model name
    model_name=arg_model_name,     # UC model name
    endpoint_name=endpoint_name,    # Clean serving name
)

In [ ]:

# ---------------------------------------------------------
# Deploy or update endpoint
# ---------------------------------------------------------
logger.info("Starting deployment/update of serving endpoint...")

model_serving.deploy_or_update_serving_endpoint(version=model_version)

logger.info("✅ Deployment/update API call completed.")


In [ ]:


# ---------------------------------------------------------
# Print serving endpoint URL
# ---------------------------------------------------------
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")

serving_url = f"https://{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

logger.info("🔥 Model successfully deployed!")
logger.info("🚀 Serving Endpoint URL:\n{}", serving_url)




In [ ]:
# ---------------------------------------------------------
# Optional: wait until endpoint is READY (CI-safe)
# ---------------------------------------------------------
# if args.wait_for_ready:
# if arg_wait_for_ready:
#     logger.info("Waiting for serving endpoint to become READY...")

#     model_serving.workspace.wait_get_serving_endpoint(
#         name=endpoint_name,
#         # timeout=args.timeout_sec
#         timeout=arg_timeout_sec,
#     )

#     logger.info("🚀 Serving endpoint is READY.")

In [ ]:
dbutils.jobs.taskValues.set(key="workspace_url", value=workspace_url)
dbutils.jobs.taskValues.set(key="endpoint_name", value=serving_url)